In [1]:
#%load_ext nb_black

In [2]:
import sys

sys.path.append("../")
import yaml
import logging
import requests
from pprint import pprint
import json
import time
import os
from pymongo import MongoClient
import dateparser
import psutil
from nlp_utils import NLPUtils
from lda_module import LdaModule

In [3]:
mongourl = "mongodb://localhost:27017/"
MONGO_CLIENT = MongoClient(mongourl)
LANG_CODE = 'de'

In [4]:
collection = MONGO_CLIENT["news"]["article" + "_" + LANG_CODE]
not_processed_docs = collection.find()
# for doc in not_processed_docs:
#    print(doc)
#    time.sleep(10)

In [5]:
training_set = []
i = 0
for document in not_processed_docs:
    if i % 10000 == 0:
        print(i)
        print(psutil.virtual_memory())
    training_set.append(document['text'])
    i += 1

0
svmem(total=8589934592, available=2425147392, percent=71.8, used=3707502592, free=990253056, active=1310035968, inactive=1120862208, wired=2397466624)
10000
svmem(total=8589934592, available=1924874240, percent=77.6, used=4042141696, free=284884992, active=1632104448, inactive=1308426240, wired=2410037248)
20000
svmem(total=8589934592, available=1841209344, percent=78.6, used=4224475136, free=50065408, active=1793085440, inactive=1447931904, wired=2431389696)
30000
svmem(total=8589934592, available=1612615680, percent=81.2, used=3968475136, free=56684544, active=1559691264, inactive=1434034176, wired=2408783872)
40000
svmem(total=8589934592, available=1392787456, percent=83.8, used=3736399872, free=52076544, active=1342169088, inactive=1244114944, wired=2394230784)
50000
svmem(total=8589934592, available=1261916160, percent=85.3, used=3602976768, free=38088704, active=1226506240, inactive=1116123136, wired=2376470528)
60000
svmem(total=8589934592, available=1267142656, percent=85.2, 

In [6]:
# Load custom stop words from file
with open('custom_stop_words.json', 'r') as sw:
    custom_stop_words = json.load(sw)
custom_stop_words = custom_stop_words['s_words']

In [7]:
nlp_utils = NLPUtils(lang=LANG_CODE, custom_stop_words = custom_stop_words)

OSError: [E050] Can't find model 'de_core_news_md'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [ ]:
# Some preparation before running LDA
doc_collection = []
print("Parsing articles...")
i = 0
for doc in training_set[:1000]:
    if i % 100 == 0: print(i)
    tokens = nlp_utils.parse_text(doc)
    doc_collection.append(tokens)
    i += 1

In [ ]:
print("Completed parsing {} articles".format(len(doc_collection)))

num_docs = len(doc_collection)
num_topics = 20
lda = LdaModule(lang = LANG_CODE, num_docs = num_docs, doc_collection = doc_collection, num_topics = num_topics, trained = False)
lda.runLDA()

docs_topics_dict = lda.get_docs_topics_dict()

In [ ]:
# == Saving model checkpoint ==
lda.save_LDA_model()